# Решение задачи о фондовом рынке

По своей сути задача является полным аналогом знаметиной задачи о рюкзаке. В данной версии задачи используются вещественные значения в качестве весов(стоимость покупки акции), поэтому эта задача решается только полным перебором всех возможных вариантов(NP полные задачи на текущий момент неразрешимы за полиномиальное время). 

Сначала я пробовал написать перебор с отсечениями с помощью рекурсии, но вскоре стало ясно что восстановление ответа будет невозможным без колоссальных затрат по памяти. Поэтому я реализовал полный перебор по маске. 

Общая асимптотика алгоритма получилась  $O(2^N * N log(N))$. $О(Nlog(N))$ уходит на проверку оптимальности ответа. 

In [1]:
from numba import njit # jit компилятор для ускорения питон-кода
import numba as nb 
import time

In [2]:
@njit # ускорение функции
def power_2_before_number(number: int) -> (int, int):
    i = 1
    k = 0
    while (i * 2 <= number):
        i *= 2
        k += 1
    return i, k

In [3]:
@njit
def solve(summ:int, prices:list, counts:list) -> (int, list):

    k = len(prices)

    number_of_masks = pow(2, k)
    ans = 0
    ans_bought_lots = nb.typed.List()
    bought_lots = nb.typed.List()
    
    for mask in range(number_of_masks):
        summ_copy = summ
        mask_copy = mask
        
        if mask == 0:
            number = 0
        else:
            number, power_2 = power_2_before_number(mask)
        ans_current = 0

        while number != 0 and summ_copy >= 0:
            summ_copy -= prices[power_2] * counts[power_2]
            if summ_copy < 0:
                break
            ans_current += counts[power_2] * 30 - (prices[power_2] - 1000) * counts[power_2]
            bought_lots.append(power_2)
            mask_copy -= number
            power_2 -= 1
            number //= 2
            
            while number > mask_copy:
                number //= 2
                power_2 -= 1
                
        if summ_copy >= 0 and ans_current > ans:
            ans = ans_current
            ans_bought_lots.clear()
            for item in bought_lots:
                ans_bought_lots.append(item)
        bought_lots.clear()

    ans_bought_lots = ans_bought_lots[::-1]

    return ans, ans_bought_lots

In [4]:
with open('data.txt', 'r') as f:
    data = f.read()
print(data)

data = data.split('\n')

25 1 8000
1 a 108.82 4
2 b 73.91 1
3 c 129.35 5
4 d 111.34 4
5 e 125.08 4
6 f 94.97 3
7 g 111.92 4
8 h 120.45 2
9 i 95.44 4
10 j 81.07 3
11 k 115.57 1
12 l 74.0 5
13 m 70.47 5
14 n 70.63 5
15 o 80.88 2
16 p 110.62 5
17 q 83.46 5
18 r 120.67 3
19 s 71.67 2
20 t 126.67 3
21 u 127.05 1
22 v 88.65 4
23 w 126.84 4
24 x 126.83 5
25 y 123.18 5


25 сэмплов более чем достаточно для проверки алгоритма. Полнопереборное решение уже здесь требует много времени

In [5]:
days = nb.typed.List()
names = nb.typed.List()
prices = nb.typed.List()
counts = nb.typed.List()

n, m, summ = map(int, data[0].split())
for req in data[1:]:
    req = req.split()
    days.append(int(req[0]))
    names.append(req[1])
    prices.append(float(req[2]))
    counts.append(int(req[3]))
    prices[-1] *= 10

start = time.time()
ans, ans_bought_lots = solve(summ, prices, counts)
end = time.time() - start

print(ans)
for i in ans_bought_lots:
    print(days[i], names[i], prices[i] // 10, counts[i])

print(f'Time: {end}')

NameError: name 'solve' is not defined

С помощью jit компилятора данный алгоритм отработал довольно быстро. Без jit этот же алгоритм работает за 111 секунд. Ускорение в 10 раз.

Приведу так же эвристический динамический алгоритм решения этой задачи. В нем я округляю цену, которую нужно заплатить за акции вверх, тем самым сводя эту задачу к подвиду рюкзака с интовыми весами. Асимптотика такого алгоритма будет $O(N*S)$
Данный алгоритм довольно точен, так как мы, по сути, отбрасываем максимум 0.1% цены лота, что часто будет приводить к оптимальному решению:


In [ ]:
from math import ceil

def solve2(x:int, summ:int, prices:list, counts:list):
    global dp
    if x + 1 == len(prices):
        return 0

    answer = dp[x][summ]
    if answer != -1:
        return answer
    answer = solve2(x + 1, summ, prices, counts)
    if ceil(prices[x] * counts[x]) <= summ:
        res = solve2(x + 1, summ - ceil(prices[x] * counts[x]), prices, counts)
        answer = max(answer, res + counts[x] * 30 - (prices[x] - 1000) * counts[x])
    dp[x][summ] = answer

    return answer

In [ ]:
dp = n * [[-1] * (summ + 1)]

start = time.time()
ans = solve2(0, summ, prices, counts)
end = time.time() - start
print(ans)
print(f'Time: {end}')

Как можно видеть, на данном тесте это решение выдает тот же ответ, что и полнопереборное, но затрачивает времени в сотни раз меньше(И это без jit компилятора). С ростом N отрыв будет все сильнее и сильнее увеличиваться.
